In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

23/05/07 21:46:13 WARN Utils: Your hostname, arkadia-355.local resolves to a loopback address: 127.0.0.1; using 10.0.1.12 instead (on interface en0)
23/05/07 21:46:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/07 21:46:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/07 21:46:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
schema = types.StructType([
    types.StructField('type', types.StringType(), True), 
    types.StructField('key', types.StringType(), True), 
    types.StructField('revision', types.IntegerType(), True), 
    types.StructField('last_modified', types.TimestampType(), True), 
    types.StructField('json', types.StringType(), True)
])

In [4]:
df = (
    spark.read
    .option("sep", "\t")
    .option("header", "false")
    .schema(schema)
    .csv("ol_dump_authors_latest.txt")
)

23/05/07 21:46:31 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [5]:
df = df.repartition(24)

In [9]:
df.select("json").write.json("ol/authors/json")

In [10]:
df_json = spark.read.json("ol/authors/json")

In [11]:
df_json.show()

+--------------------+
|                json|
+--------------------+
|{"name": "Kris Ri...|
|{"name": "S. M. N...|
|{"type": {"key": ...|
|{"name": "Germain...|
|{"type": {"key": ...|
|{"type": {"key": ...|
|{"type": {"key": ...|
|{"name": "Max Hay...|
|{"name": "Maria-J...|
|{"type": {"key": ...|
|{"name": "Nam-mkh...|
|{"type": {"key": ...|
|{"name": "Jacques...|
|{"name": "Joe\u03...|
|{"name": "Emilia ...|
|{"name": "Ishraga...|
|{"name": "Hilda P...|
|{"type": {"key": ...|
|{"name": "Michael...|
|{"name": "Symposi...|
+--------------------+
only showing top 20 rows

